# Análisis de elecciones 2012

Se importa la libraría json

In [1]:
import json

Se procede a abrir el archivo de nombres de los archivos JSON



In [2]:
file_nombre_archivos = open('nombre_archivos.txt')

votos_totales = {}
votos_totales_revisados = {}
porcentaje={}
votos_totales_entidad = {str(i):{} for i in range(1, 33)}
votos_totales_entidad_revisados = {str(i):{} for i in range(1, 33)}
estados = {'1': 'Aguascalientes', '2': 'Baja California', '3': 'Baja California Sur',
           '4': 'Campeche', '5': 'Coahuila', '6': 'Colima', '7': 'Chiapas', '8': 'Chihuahua',
           '9': 'Ciudad de México', '10': 'Durango', '11': 'Guanajuato', '12': 'Guerrero',
           '13': 'Hidalgo', '14': 'Jalisco', '15': 'México', '16': 'Michoacán', '17': 'Morelos',
           '18': 'Nayarit', '19': 'Nuevo León', '20': 'Oaxaca', '21': 'Puebla', '22': 'Querétaro',
           '23': 'Quintana Roo', '24': 'San Luis Potosi', '25': 'Sinaloa', '26': 'Sonora',
           '27': 'Tabasco', '28': 'Tamaulipas', '29': 'Tlaxcala', '30': 'Veracruz',
           '31': 'Yucatán', '32': 'Zacatecas'}
#Abrir cada archivo
for archivo in file_nombre_archivos:    
    archivo = archivo.rstrip()
    data = json.load(open(archivo))
    #Abrir cada sabana
    for sabana in data['sabanas']:
        #Contar votos revisados almenos 2 veces         
        if int(sabana['revision']) >= 2:
            for partido in sabana['resultados'].keys():       
                votos_totales_entidad_revisados[sabana['entidad']][partido] = votos_totales_entidad_revisados[sabana['entidad']].get(partido, 0) + int(sabana['resultados'][partido])
            for partido in sabana['resultados'].keys():
                votos_totales_revisados[partido]=votos_totales_revisados.get(partido, 0)+int(sabana['resultados'][partido])
        #Contar los votos por Entidad
        for partido in sabana['resultados'].keys():       
            votos_totales_entidad[sabana['entidad']][partido] = votos_totales_entidad[sabana['entidad']].get(partido, 0) + int(sabana['resultados'][partido])
        #Contar los votos por partido
        for partido in sabana['resultados'].keys():       
            votos_totales[partido] = votos_totales.get(partido, 0) + int(sabana['resultados'][partido])

In [3]:
# Separar los votos por partidos de los de coalicion
votos_partidos = {p: v for p, v in votos_totales.items() if '_' not in p}
votos_coalicion = {c: v for c, v in votos_totales.items() if '_' in c}

votos_partidos_revisados = {p: v for p, v in votos_totales_revisados.items() if '_' not in p}
votos_coalicion_revisados = {c: v for c, v in votos_totales_revisados.items() if '_' in c}

votos_partidos_entidad = {entidad:{p: v for p, v in votos_totales_entidad[entidad].items() if '_' not in p} for entidad in votos_totales_entidad.keys()}
votos_coalicion_entidad = {entidad:{c: v for c, v in votos_totales_entidad[entidad].items() if '_' in c} for entidad in votos_totales_entidad.keys()}

votos_partidos_entidad_revisados = {entidad:{p: v for p, v in votos_totales_entidad_revisados[entidad].items() if '_' not in p} for entidad in votos_totales_entidad.keys()}
votos_coalicion_entidad_revisados = {entidad:{c: v for c, v in votos_totales_entidad_revisados[entidad].items() if '_' in c} for entidad in votos_totales_entidad.keys()}

## Votos por Partidos

In [4]:
# Resultados de la gráfica de Votos por Partidos
def resultados_votos_partido(votos_p, votos_c):
    resultados_partido = {}
    for partido, voto_partido in votos_p.items():
        resultados_partido[partido] = voto_partido
        for coalicion, voto_c in votos_c.items():
            if partido in coalicion:
                resultados_partido[partido] = resultados_partido.get(partido, 0) + voto_c // (coalicion.count('_') + 1)
                max_votos_partido = -1
                max_partido = ''
                for p in coalicion.split('_'):
                    if votos_p[p] > max_votos_partido:
                        max_votos_partido = votos_p[p]
                        max_partido = p
                if partido == max_partido:
                    resultados_partido[partido] = resultados_partido.get(partido, 0) + voto_c % (coalicion.count('_') + 1)
    return resultados_partido

In [5]:
resultados_partido = resultados_votos_partido(votos_partidos, votos_coalicion)
resultados_partido_revisados = resultados_votos_partido(votos_partidos_revisados, votos_coalicion_revisados)
resultados_partido_entidad = {entidad: resultados_votos_partido(votos_partidos_entidad[entidad], votos_coalicion_entidad[entidad]) for entidad in votos_totales_entidad.keys()}
resultados_partido_entidad_revisados = {entidad: resultados_votos_partido(votos_partidos_entidad_revisados[entidad], votos_coalicion_entidad_revisados[entidad]) for entidad in votos_totales_entidad.keys()}

## Votos por Candidatos

In [6]:
def resultados_votos_candidatos(votos_p, votos_c):
    # Resultados de la gráfica de Votos por Candidatos
    resultados_candidato = {}
    for coal_1 in votos_c:
        for coal_2 in votos_c:
            if coal_1 != coal_2 and not set(coal_1.split('_')) - set(coal_2.split('_')):
                break
        else:
            resultados_candidato[coal_1] = 0

    # Agragar los votos de las coaliciones a los candidatos
    for subcoalicion, subvotos in votos_c.items():
        for coalicion in resultados_candidato.keys():
            if subcoalicion[:subcoalicion.find('_')] in coalicion:
                resultados_candidato[coalicion] = resultados_candidato.get(coalicion, 0) + subvotos

    # Agragar los votos de los partidos a los candidatos
    tmp_resultados_candidato = [i for i in resultados_candidato.keys()]
    for partido, votos in votos_p.items(): 
        for coalicion in tmp_resultados_candidato:
            if partido in coalicion:
                resultados_candidato[coalicion] = resultados_candidato.get(coalicion, 0) + votos
                break
        else:
            resultados_candidato[partido] =  votos
    return resultados_candidato

In [7]:
resultados_candidato = resultados_votos_candidatos(votos_partidos, votos_coalicion)
resultados_candidato_revisados = resultados_votos_candidatos(votos_partidos_revisados, votos_coalicion_revisados)
resultados_coalision_entidad = {entidad: resultados_votos_candidatos(votos_partidos_entidad[entidad], votos_coalicion_entidad[entidad]) for entidad in votos_totales_entidad.keys()}
resultados_coalision_entidad_revisados = {entidad: resultados_votos_candidatos(votos_partidos_entidad_revisados[entidad], votos_coalicion_entidad_revisados[entidad]) for entidad in votos_totales_entidad.keys()}

# Resultados

# Revisión vs no revisión por partido nacionalmente

In [8]:
print("Partidos no revisados nacional")
for k, v in resultados_partido.items():
    print(f'{k:25}{v}')
print("-"*30)
print("Partidos revisados nacional")
for k, v in resultados_partido_revisados.items():
    print(f'{k:25}{v}')

Partidos no revisados nacional
pan                      2247447
pri                      2341599
prd                      2601010
verde                    404150
pt                       592917
mov                      503683
nuevaAlianza             203491
noRegistrados            10632
nulos                    163943
------------------------------
Partidos revisados nacional
pan                      2178205
pri                      2260913
prd                      2522078
verde                    390090
pt                       575307
mov                      488774
nuevaAlianza             196698
noRegistrados            9342
nulos                    157324


# Revisión vs no revisión por candidato nacionalmente

In [9]:
print("Candidatos no revisados nacional")
for k, v in resultados_candidato.items():
    print(f'{k:25}{v}')
print("-"*30)
print("Candidatos revisados nacional")
for k, v in resultados_candidato_revisados.items():
    print(f'{k:25}{v}')

Candidatos no revisados nacional
pri_verde                2745749
prd_pt_mov               3697610
pan                      2247447
nuevaAlianza             203491
noRegistrados            10632
nulos                    163943
------------------------------
Candidatos revisados nacional
pri_verde                2651003
prd_pt_mov               3586159
pan                      2178205
nuevaAlianza             196698
noRegistrados            9342
nulos                    157324


# Resultados de todos los votos a partidos por entidad

In [10]:
print("Entidad", end=" "*20)
for k in resultados_partido_entidad["1"].keys():
    print(f"{k:10}", end="")
print("Suma")
print()
for entidad, diccionario in resultados_partido_entidad.items():
    print(f"{estados[entidad]:20}", end="")
    for resultado in diccionario.values():
        print(f"{resultado:10}", end = "")
    print("     ",sum(diccionario.values()))
    print()

Entidad                    pan       pri       prd       verde     pt        mov       nuevaAlianzanoRegistradosnulos     Suma

Aguascalientes           22048     20314     12431      3258      2807      2667      3965        94      1926      69510

Baja California         127975    123625    115110     21052     25490     21513     12460       271      7141      454637

Baja California Sur      12269     13321      7445      2140      2017      1852       811        21       608      40484

Campeche                 10400     11940     10505      1956      2343      2860      1110        17       807      41938

Coahuila                 51554     35523     23310      4473      5000      3753      3459        79      1876      129027

Colima                   16177     14286      8509      1535      2193      1459       796        12       601      45568

Chiapas                  18082     26897     36988     15587      8777      7332      2607        52      3467      119789

Chihuahu

# Resultados de votos revisados a partidos por entidad

In [11]:
print("Entidad", end=" "*20)
for k in resultados_partido_entidad_revisados["1"].keys():
    print(f"{k:10}", end="")
print("Suma")
print()
for entidad, diccionario in resultados_partido_entidad_revisados.items():
    print(f"{estados[entidad]:20}", end="")
    for resultado in diccionario.values():
        print(f"{resultado:10}", end = "")
    print("     ",sum(diccionario.values()))
    print()

Entidad                    pan       pri       prd       verde     pt        mov       nuevaAlianzanoRegistradosnulos     Suma

Aguascalientes           21066     19440     11818      3017      2653      2536      3760        92      1835      66217

Baja California         123652    119102    111273     20299     24652     20801     12047       257      6519      438602

Baja California Sur      12183     13239      7409      2127      2009      1842       805        21       603      40238

Campeche                 10290     11739     10416      1942      2293      2802      1086        17       801      41386

Coahuila                 49341     33846     22322      4272      4791      3606      3299        77      1805      123359

Colima                   15895     14029      8372      1508      2164      1443       786        12       596      44805

Chiapas                  17491     25992     35967     15104      8530      7140      2554        52      3380      116210

Chihuahu

# Resultados de todos los votos a candidatos por entidad

In [12]:
print("Entidad", end=" "*20)
for k in resultados_coalision_entidad["1"].keys():
    print(f"{k:15}", end="")
print("Suma")
print()
for entidad, diccionario in resultados_coalision_entidad.items():
    print(f"{estados[entidad]:20}", end="")
    for resultado in diccionario.values():
        print(f"{resultado:15}", end = "")
    print("     ",sum(diccionario.values()))
    print()

Entidad                    pri_verde      prd_pt_mov     pan            nuevaAlianza   noRegistrados  nulos          Suma

Aguascalientes                23572          17905          22048           3965             94           1926      69510

Baja California              144677         162113         127975          12460            271           7141      454637

Baja California Sur           15461          11314          12269            811             21            608      40484

Campeche                      13896          15708          10400           1110             17            807      41938

Coahuila                      39996          32063          51554           3459             79           1876      129027

Colima                        15821          12161          16177            796             12            601      45568

Chiapas                       42484          53097          18082           2607             52           3467      119789

Chihuahua    

# Resultados de los votos revisados a candidatos por entidad

In [13]:
print("Entidad", end=" "*20)
for k in resultados_coalision_entidad_revisados["1"].keys():
    print(f"{k:15}", end="")
print("Suma")
print()
for entidad, diccionario in resultados_coalision_entidad_revisados.items():
    print(f"{estados[entidad]:20}", end="")
    for resultado in diccionario.values():
        print(f"{resultado:15}", end = "")
    print("     ",sum(diccionario.values()))
    print()

Entidad                    pri_verde      prd_pt_mov     pan            nuevaAlianza   noRegistrados  nulos          Suma

Aguascalientes                22457          17007          21066           3760             92           1835      66217

Baja California              139401         156726         123652          12047            257           6519      438602

Baja California Sur           15366          11260          12183            805             21            603      40238

Campeche                      13681          15511          10290           1086             17            801      41386

Coahuila                      38118          30719          49341           3299             77           1805      123359

Colima                        15537          11979          15895            786             12            596      44805

Chiapas                       41096          51637          17491           2554             52           3380      116210

Chihuahua    